In [435]:
import arcgis
from arcgis.apps import storymap
from arcgis.features import FeatureLayerCollection

In [436]:
import arcgis
from arcgis.gis import GIS
from arcgis.apps import storymap
from getpass import getpass
from arcgis.features import FeatureLayer

# Create a connection to your portal for publishing (enter your ArcGIS Online 
#  password in the textbox that appears, then hit 'Enter' on your keyboard)
gis = GIS("https://www.arcgis.com", "ezhatcher", getpass())

········


In [437]:
seedappItem = gis.content.get("2a51cb9cc9d54b57a52c67d8efdb5eba") #Hoonah Alaska Template App

In [438]:
contentManager = arcgis.gis.ContentManager(gis)
clones = contentManager.clone_items(items=[seedappItem],search_existing_items=False)
clones
clonedStorymap = storymap.JournalStoryMap(clones[0])

In [439]:
community = 'Angoon'
clonedStorymap.save(title=str(community) +" Storymap Test", description="Test cloning script result", tags="Test,Storymap")
sections = clonedStorymap.properties["values"]["story"]["sections"]

In [440]:
def access_lyr(content,out_fld):
    base = content.layers
    lyr = base[0]
    topic_data = lyr.query(where="CommunityName = '" + str(community) + "'",out_fields=out_fld)
    return topic_data

In [441]:
#Access the URL for the main photo from the Photo Link feature class
photo_base = gis.content.get("6d9d437bbe2d48bfa87b4d9fca41f80d")
find_photo = access_lyr(photo_base,'PhotoLink')
photo_url = find_photo.df
main_pic = photo_url.PhotoLink[0]

In [442]:
#Identify the panels
panel_toc = sections[0]
panel_ch = sections[1]
panel_demog = sections[2]
panel_transport = sections[3]
panel_edu = sections[4]
panel_econ = sections[5]
panel_util = sections[6]
panel_ANCSA = sections[7]
panel_contacts = sections[8]

panel_list = [panel_ch, panel_demog, panel_transport, panel_edu, panel_econ, panel_util, panel_ANCSA, panel_contacts]


#Modify the main title and photo in the main Table of Contents panel
panel_toc["title"] = '<span style="font-size:40px">' + str(community) + ', Alaska</span>'
panel_toc["media"]["image"]["url"] = main_pic
clonedStorymap.save()

True

In [443]:
#Set up zoom scale (x max and min, y max and min) for map actions
bound_expr = "NAME = '" + str(community) + "'"
bounds_ft = gis.content.get("8499c3bc4ff141588f5c9ab1db99b0fb").layers[0]
x_min = bounds_ft.query(where=bound_expr,out_fields='EXT_MIN_X').df.EXT_MIN_X[0]
x_max = bounds_ft.query(where=bound_expr,out_fields='EXT_MAX_X').df.EXT_MAX_X[0]
y_min = bounds_ft.query(where=bound_expr,out_fields='EXT_MIN_Y').df.EXT_MIN_Y[0]
y_max = bounds_ft.query(where=bound_expr,out_fields='EXT_MAX_Y').df.EXT_MAX_Y[0]

#Function to define the primary zoom scale of each panel based on the minimum bounding envelope feature service
# item id = "8499c3bc4ff141588f5c9ab1db99b0fb" using previously defined zoom scale
def main_extent(p):
    ext1 = p["media"]["webmap"]["extent"]
    ext1["xmin"] = x_min
    ext1["xmax"] = x_max
    ext1["ymin"] = y_min
    ext1["ymax"] = y_max
    clonedStorymap.save()

In [444]:
#Use the demographics panel to access the webmap and clone the webmap/update map filters
panel_demog = sections[2]
current_webmap = gis.content.get(panel_demog["contentActions"][1]["media"]["webmap"]["id"])
wm = arcgis.mapping.WebMap(webmapitem=current_webmap)

#Set up filters in the web map to fit the community (IF LAYERS ARE ADDED OR REMOVED, UPDATE THIS CELL)
def_expr = "CommunityName = '" + str(community) + "'"
def_expr2 = "City = '" + str(community) + "'"
conv2_lyrs = [9, 10, 12]
for i in range(7):
    wm_lyrs = wm.layers[i]
    wm_lyrs["layerDefinition"]["definitionExpression"] = def_expr
panel_wm.layers[8]['layerDefinition']['definitionExpression'] = def_expr2
for n in conv2_lyrs:
    wm_lyrs = wm.layers[n]
    wm_lyrs["layers"][0]["layerDefinition"]["definitionExpression"] = def_expr
wm.update()

wm_properties = {'title':'OpenData_Storymap_' + str(community),'snippet':'Test webmap clone','tags':['dcra','test']}
wm.save(wm_properties,folder='Storymap Webmaps') 

<Item title:"OpenData_Storymap_Angoon" type:Web Map owner:ezhatcher>

In [445]:
#Change the webmap in the demographics panel
new_map_search = gis.content.search('OpenData_Storymap_' + str(community), item_type="Web Map")
webmap = new_map_search[0]
new_webmap = webmap.id
#Define the webmap item to access and filter layers
upd_webmap = gis.content.get(new_webmap)
panel_wm = arcgis.mapping.WebMap(webmapitem=upd_webmap)

In [446]:
#Update all of the panel webmaps using the new webmap ID
panel_demog["contentActions"][1]["media"]["webmap"]["id"] = new_webmap
clonedStorymap.save()

True

In [447]:
#Adjust community point y-offset (main overview map after the TOC panel)
panel_wm.layers[6]["layerDefinition"]["drawingInfo"]["renderer"]["symbol"]["yoffset"] = 16
panel_wm.update()

True

In [449]:
#Access the culture and history from the culture/history feature class (IF THE PANEL ORDER CHANGES, MAKE SURE INDEX STILL APPLIES)
panel_ch = sections[1]
ch_ft = gis.content.get("3f99cf21a0da42c4bf8b15632e958df9")
culture_lookup = access_lyr(ch_ft,'CommunityCulture')
culture = culture_lookup.df.CommunityCulture[0]
history_lookup = access_lyr(ch_ft,'CommunityHistory')
history = history_lookup.df.CommunityHistory[0]

In [450]:
#Populate the panel with the new culture history info
panel_ch["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;<p>'+ history + '&nbsp;</p>\n\n<p>\n\n<p>&nbsp;<p>'+ culture + '&nbsp;</p>\n'
panel_ch["media"]["webmap"]["id"] = new_webmap
clonedStorymap.save()

True

In [451]:
#Demographics Panel Updates
#Update the zoom extent for the demographics panel map action (tied to 'Historic Census' button)
panel_demog["media"]["webmap"]["id"] = new_webmap
lyr_inmap = panel_demog["media"]["webmap"]["popup"]["layerId"]
lyr_search = gis.content.search(lyr_inmap[:-4], item_type="Feature Service")
DCCED_lyr = gis.content.get(lyr_search[0].id)
DCCED_oid = DCCED_lyr.layers[0].query(where="CommunityName = '" + str(community) + "'",out_fields='CommunityName')
DCCED_oid.df.ESRI_OID[0]
DCCED_geom = DCCED_oid.df.SHAPE
DCCED_x = DCCED_geom[0]["x"]
DCCED_y = DCCED_geom[0]["y"]
community_pop = DCCED_oid.df.CommunityName[0]

In [452]:
###DEMOGRAPHICS PANEL###
#Update map and popup extents
demog_mainpopup = panel_demog["media"]["webmap"]["popup"]
demog_mainpopup["fieldName"] = 'CommunityName'
demog_mainpopup["fieldValue"] = str(community_pop)
demog_mainpopup["anchorPoint"]["x"] = DCCED_x
demog_mainpopup["anchorPoint"]["y"] = DCCED_y

main_extent(panel_demog)

#Update the main demographics panel popup
demog_histpopup = panel_demog["contentActions"][1]["media"]["webmap"]["popup"]
demog_histext = panel_demog["contentActions"][1]["media"]["webmap"]["extent"]

#Update historic census map action extent
demog_histext["xmin"] = x_min
demog_histext["xmax"] = x_max
demog_histext["ymin"] = y_min
demog_histext["ymax"] = y_max

#Update historic census popup data and extent
demog_histpopup["fieldName"] = 'CommunityName'
demog_histpopup["fieldValue"] = str(community_pop)
demog_histpopup["x"] = DCCED_x
demog_histpopup["y"] = DCCED_y
demog_histpopup["anchorPoint"]["x"] = DCCED_x
demog_histpopup["anchorPoint"]["y"] = DCCED_y
clonedStorymap.save()

True

In [453]:
###TRANSPORTATION PANEL###
#Update main map extent
main_extent(panel_transport)


#Popup extents...
#panel_transport["contentActions"][1]["media"]["webmap"]["extent"]["xmin"] = x_min
#panel_transport["contentActions"][1]["media"]["webmap"]["extent"]["xmax"] = x_max
#panel_transport["contentActions"][1]["media"]["webmap"]["extent"]["ymin"] = y_min
#panel_transport["contentActions"][1]["media"]["webmap"]["extent"]["ymax"] = y_max
clonedStorymap.save()

True

In [454]:
#TRANSPORTATION PANEL#
#Update panel text using transportation overview feature class (item id = "2076ce8b6744430880e1c8ab923cd720")
find_text = gis.content.get("2076ce8b6744430880e1c8ab923cd720")
access_text = find_text.layers[0].query(where="CommunityName = '" + str(community) + "'",out_fields='Description').df
panel_transport["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 3px 8px;\n     border-radius: 5px;\n}\n</style>\n<style type="text/css">.btn-orange {\n    display: inline-block;\n    background-color: #f0ad4e;\n    border-color: #f0ad4e !important;\n    color: #fff !important;\n    padding: 0px 2px;\n    border-radius: 12px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524784108786" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<p>' + access_text.Description[0] + '</p>\n\n<p>&nbsp;</p>\n\n<p>More detailed information about Transportation in Alaska can be found <a href="http://DCCED.maps.arcgis.com/apps/webappviewer/index.html?id=3de3ebccbb6b4ba8a9c64609e96199ba" target="_blank"><strong>here</strong></a>.&nbsp;</p>\n'
clonedStorymap.save()

True

In [455]:
###EDUCATION PANEL###
#Add community webmap to panel
panel_edu["media"]["webmap"]["id"] = new_webmap

#Update extent
main_extent(panel_edu)



#Popups if we can iteratively create map actions per school in each community...
#panel_edu["contentActions"][1]["media"]["webmap"]["id"] = new_webmap
#popup_ext = panel_edu["contentActions"][1]["media"]["webmap"]["extent"]
#popup_ext["xmin"] = x_min
#popup_ext["xmax"] = x_max
#popup_ext["ymin"] = y_min
#popup_ext["ymax"] = y_max
clonedStorymap.save()

True

In [456]:
###EDUCATION PANEL###
#Update panel schools listing
sch_ref = panel_edu["contentActions"][1]["media"]["webmap"]["popup"]["layerId"]
find_sch_lyr = gis.content.search("Alaska PK-12", item_type = "Map Service")
schools = find_sch_lyr[0].layers[0].query(where="CommunityName = '" + str(community) + "'",out_fields='EntityName')
school_dist = find_sch_lyr[0].layers[0].query(where="EntityName = '" + str(schools.df.EntityName[0]) + "'", out_fields='SchoolDistrict').df.SchoolDistrict[0]
school_lst = []
for school in schools.df.EntityName:
    school_lst.append(school)

## NEXT, FIGURE OUT HOW TO ITERATIVELY CREATE MAP ACTIONS FOR EACH SCHOOL

In [457]:
panel_edu["content"] = '<p>&nbsp;</p>\n\n<p>\n<style type="text/css">.blue {\n     display: inline-block;\n     background-color: #3182bd;\n     border-color: #3182bd !important;\n     color: #fff !important;\n     padding: 1px 3px;\n     border-radius: 12px;\n</style>\n<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 3px 8px;\n     border-radius: 5px;\n</style>\n</p>\n<style type="text/css">.btn-orange {\n    display: inline-block;\n    background-color: #f0ad4e;\n    border-color: #f0ad4e !important;\n    color: #fff !important;\n    padding: 0px 2px;\n    border-radius: 12px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524785467480" data-storymaps-type="navigate">Back to Table of Contents</a><br>\n<style type="text/css">\n</style>\n</p>\n\n<p>&nbsp;</p>\n\n<p>&nbsp;</p>\n\n<p>&nbsp;</p>\n\n<p>' + str(community) + ' schools are part of ' + str(school_dist) + '. </p>\n\n<ul>\n\t'

clonedStorymap.save()

True

In [458]:
###ECONOMICS PANEL###
#Update map extent
main_extent(panel_econ)

clonedStorymap.save()

True